In [4]:
import tensorflow as tf
filename_queue = tf.train.string_input_producer(["data/file0.csv", "data/file1.csv"])

reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

# Default values, in case of empty columns. Also specifies the type of the
# decoded result.
record_defaults = [[1], [1], [1], [1], [1], [1]]
col1, col2, col3, col4, col5, col6 = tf.decode_csv(
    value, record_defaults=record_defaults)
features = tf.pack([col1, col2, col3, col4, col5])

with tf.Session() as sess:
  # Start populating the filename queue.
  coord = tf.train.Coordinator()
  threads = tf.train.start_queue_runners(coord=coord)

  for i in range(10):
    # Retrieve a single instance:
    example, label = sess.run([features, col6])
    print (example, label)

  coord.request_stop()
  coord.join(threads)

[1 2 3 4 5] 6
[1 2 3 4 5] 6
[1 2 3 4 5] 6
[1 2 3 4 5] 6
[1 2 3 4 5] 6
[1 2 3 4 5] 6
[1 2 3 4 5] 6
[1 2 3 4 5] 6
[1 2 3 4 5] 6
[1 2 3 4 5] 6


In [5]:
def read_my_file_format(filename_queue):
  reader = tf.SomeReader()
  key, record_string = reader.read(filename_queue)
  example, label = tf.decode_csv(record_string)
  #processed_example = some_processing(example)
  return example, label

def input_pipeline(filenames, batch_size, num_epochs=None):
  filename_queue = tf.train.string_input_producer(
      filenames, num_epochs=num_epochs, shuffle=True)
  example, label = read_my_file_format(filename_queue)
  # min_after_dequeue defines how big a buffer we will randomly sample
  #   from -- bigger means better shuffling but slower start up and more
  #   memory used.
  # capacity must be larger than min_after_dequeue and the amount larger
  #   determines the maximum we will prefetch.  Recommendation:
  #   min_after_dequeue + (num_threads + a small safety margin) * batch_size
  min_after_dequeue = 10000
  capacity = min_after_dequeue + 3 * batch_size
  example_batch, label_batch = tf.train.shuffle_batch(
      [example, label], batch_size=batch_size, capacity=capacity,
      min_after_dequeue=min_after_dequeue)
  return example_batch, label_batch

input_pipeline("data/file0.csv", 10)

ValueError: Shape () must have rank at least 1